In [1]:
try:
    from tqdm import tqdm
except ImportError:
    def tqdm(iterator, *args, **kwargs):
        return iterator
import os, sys, glob, warnings, glob    
import numpy as np
import scipy as sp
from scipy import constants
from pylab import *
import joblib
import importlib
from importlib import reload
sys.path.append("/home/owhgabri/My_GitHub/pyTracker/tracker")
sys.path.insert(1, "/home/owhgabri/My_GitHub/pyTracker/tracker")
os.chdir('/home/owhgabri/My_GitHub/pyTracker')
print(os.getcwd())
print(joblib.__version__)

/home/owhgabri/My_GitHub/pyTracker
1.4.2


In [2]:
import scipy
import copy as cp

# ROOT
import ROOT as root

# Matplotlib
import matplotlib.pyplot as plt
from matplotlib import collections, colors, transforms

%matplotlib inline
%config InlineBackend.figure_format='retina'
# %matplotlib widget

import pprint

Welcome to JupyROOT 6.26/14


In [3]:
import kalmanfilter as KF
import utilities as Util
import trackfinder as TF
import datatypes
from datatypes import *

reload(TF)
reload(Util)

<module 'utilities' from '/home/owhgabri/My_GitHub/pyTracker/tracker/utilities.py'>

# Defining Useful Functions #

In [4]:
x_mod_starts = [-1950, -950, 50, 950, 1050]
y_layer_starts = [8550, 8631.6, 9730, 9811.6, 9893.2, 9974.8, 10056.4, 10138]
n_floors = 2
z_mod_starts = [7000, 8000, 9000, 10000]
z_wall_starts = [10900, 10981.6, 11063.2, 11144.8, 11226.4, 11308]
back_top =  y_layer_starts[4]
beam_dim = 10
mod_length = 900
front_wall_starts = [6895.8,6997.4]
steel_height=3 # cm
Box_IP_Depth=8547 # cm   
c = 29.979

wall_floor1 = (7000,8633.2)
wall_floor2 = (7000,8551.6)
wallMid1 = 6896.6
wallMid2 = 6998.2
floorMid1 = 8550.8
floorMid2 = 8632.4

IP = (0,0,0)

y_bottoms = [8550, 8631.6]
z_fronts = [6895.8, 6997.4]
x_lims = (-1950, 1950)
thickness = 1.6

In [5]:
def SortByTime(points):
    """
    Sorts list hits by time
    A hit is of the form (x,y,z,t)
    """
    if len(points) <= 1:
        return points
    pivot = points[len(points) // 2]
    left = []
    middle = []
    right = []
    for point in points:
        if point[-1] < pivot[-1]:
            left.append(point)
        elif point[-1] > pivot[-1]:
            right.append(point)
        elif point[-1] == pivot[-1]:
            middle.append(point)
    return SortByTime(left) + middle + SortByTime(right)


def ProjectionTime(track, points, layer):
    """
    Takes a track and its SORTED list of points
    closest is true if you want to check in the closest wall/floor,
    false if you wanna check in the farther wall/floor
    Returns the time at which the track is 
    projected to enter the wall or floor
    Given point is (x,y,z,t)
    """
    if layer == 0:# Walls
        if track.Ay == 1: # Horizontal layer track 
            return track.At/track.Az*(wallMid1 - points[0][2]) + points[0][3]
        else: # Vertical layer track
            return track.At*(wallMid1 - points[0][2]) + points[0][3]
    elif layer == 1:
        if track.Ay == 1: # Horizontal layer track
            return track.At/track.Az*(wallMid2 - points[0][2]) + points[0][3]
        else: # Vertical layer track
            return track.At*(wallMid2 - points[0][2]) + points[0][3]
    elif layer == 2: # Floors
        if track.Ay == 1: # Horizontal layer track
            return track.At*(floorMid1 - points[0][1]) + points[0][3]
        else: # Vertical layer track
            return track.At/track.Ay*(floorMid1 - points[0][1]) + points[0][3]
    elif layer == 3: 
        if track.Ay == 1: # Horizontal layer track
            return track.At*(floorMid2 - points[0][1]) + points[0][3]
        else: # Vertical layer track
            return track.At/track.Ay*(floorMid2 - points[0][1]) + points[0][3]


def ProjectionPoint(track, points, t):
    """
    Returns where the track is projected to be at time t.
    points is the list of hits from the track sorted by time
    Returns an empty array if out of the detector
    """
    if track.Ay == 1: #Horizontal layer track
        xf = points[0][0] + track.Ax/track.At*(t - points[0][3])
        yf = points[0][1] + 1/track.At * (t - points[0][3])
        zf = points[0][2] + track.Az/track.At*(t - points[0][3])
    else: # Vertical layer track 
        xf = points[0][0] + track.Ax/track.At*(t - points[0][3])
        yf = points[0][1] + track.Ay/track.At * (t - points[0][3])
        zf = points[0][2] + 1/track.At*(t - points[0][3])
    return np.array((xf,yf,zf))


def GetDistance(point1, point2):
    """
    Get the distance between two points (x,y,z)
    """
    return np.sqrt((point1[0]-point2[0])**2 + (point1[1]-point2[1])**2 + (point1[2]-point2[2])**2)


def Ellipse(x, y, c1, c2, a, b):
    """
    Returns whether (x,y) lies within the ellipse
    """
    return (x-c1)**2/a**2 + (y-c2)**2/b**2


def ConvertVelocity(track):
    """
    Converts the velocity from either dk/dy or dk/dz
    to dk/dt
    """
    if track.Ay == 1: # Horizontal layer track
        A_1x = track.Ax/track.At; A_1y = 1/track.At; A_1z = track.Az/track.At
    else: # Vertical layer track
        A_1x = track.Ax/track.At; A_1y = track.Ay/track.At; A_1z = 1/track.At
    return np.array((A_1x, A_1y, A_1z))


def MaxTracks(vertex, tracks):
    """
    Get the track indices that have the largest angle of separation
    """
    inds = None
    min_angle = None
    for i in range(len(vertex.tracks)):
        ind_1 = vertex.tracks[i]
        t1 = tracks[ind_1]
        A1 = ConvertVelocity(t1)
        for j in range(i + 1, len(vertex.tracks)):
            ind_2 = vertex.tracks[j]
            t2 = tracks[ind_2]
            A2 = ConvertVelocity(t2)
            angle = np.arccos(np.dot(A1, A2)/(np.linalg.norm(A1)*np.linalg.norm(A2)))
            if min_angle is None or angle < min_angle:
                min_angle = angle
                inds = (ind_1, ind_2)
    return inds


def ClosestApproach(point, track):
    """
    Gets the point at which the track is closest to point
    """
    A = ConvertVelocity(track)
    P_0 = np.array((track.x0, track.y0, track.z0))
    t_c = np.dot((point - P_0), A)/np.linalg.norm(A)**2 + track.t0
    return P_0 + A*(t_c - track.t0)


def EllipseAngle(p1, p2):
    """
    Get the angle of the ellipse. This is always with 
    respect to the x axis
    """
    if p1[1] - p2[1] < 0.001: # Floor
        dz = p2[2] - p1[2]; dx = p2[0] - p1[0]
        slope = dz/dx
        angle = np.arctan(dz/dx)
    else: # Wall
        dy = p2[1] - p1[1]; dx = p2[0] - p1[0]
        slope = dy/dx
        angle = np.arctan(dy/dx)        
    return angle
        

def GetEllipse(vertex, tracks, layer):
    """
    Gets the center of the backwards cone of the vertex
    """
    
    ind_1, ind_2 = MaxTracks(vertex, tracks)
    t1 = tracks[ind_1]; t2 = tracks[ind_2]
    t1Points = SortByTime(t1.hits_filtered); t2Points = SortByTime(t2.hits_filtered)
    time1 = ProjectionTime(t1, t1Points, layer); time2 = ProjectionTime(t2, t2Points, layer)
    p1 = ProjectionPoint(t1, t1Points, time1); p2 = ProjectionPoint(t2, t2Points, time2)
    midpoint = (p1 + p2)/2
    c1 = midpoint[0]
    if layer < 2: # Wall
        c2 = midpoint[1]
        a = GetDistance(p1, midpoint) # Axis 1
        b = GetDistance(ClosestApproach(midpoint, t1), midpoint) # Axis 2
        angle = EllipseAngle(p1, p2)
        def Ellipse(x, y):
            """
            Returns whether (x,y) lies within the ellipse
            """
            x_p = (x-c1)*np.cos(angle) + (y-c2)*np.sin(angle)
            y_p = (y-c2)*np.cos(angle) - (x-c1)*np.sin(angle)
            return ((x_p)**2/a**2 + (y_p)**2/b**2) <= 1
    else: # Floor
        c2 = midpoint[2]
        a = GetDistance(p1, midpoint) # Axis 1
        b = GetDistance(ClosestApproach(midpoint, t1), midpoint) # Axis 2
        angle = EllipseAngle(p1, p2)
        def Ellipse(x, z):
            """
            Returns whether (x,z) lies within the ellipse
            """
            x_p = (x-c1)*np.cos(angle) + (z-c2)*np.sin(angle)
            z_p = (z-c2)*np.cos(angle) - (x-c1)*np.sin(angle)
            return ((x_p)**2/a**2 + (z_p)**2/b**2) <= 1
    return Ellipse
        
def Ellipses(vertex, tracks):
    """
    Gets the Ellipses for each layer
    """
    ells = []
    for i in range(len(y_bottoms) + len(z_fronts)):
        ells.append(GetEllipse(vertex,tracks,i))
    return ells


# Background #

In [6]:
data_top_dir = f"/home/owhgabri/My_GitHub/data/Reconstruction/10xCosmicBackground"
pathList=[]

for rootFile, dirs, files in os.walk(data_top_dir):
    for filename in files:
        if ".pkl" in filename:
            pathList.append(os.path.join(rootFile, filename))


print(len(pathList))

187


## Survival Rate: Vertex Level ##

In [7]:
n_vetoed = 0
nVertices = 0
for f in pathList:
    events=joblib.load(f)
    file_hits = events["hits"]
    file_tracks = events["tracks"]
    file_vertices = events["vertices"]
    for i in range(len(file_hits)):
        hits= file_hits[i]
        tracks = file_tracks[i]
        vertices = file_vertices[i]
        if len(vertices) == 0:
            continue
        nVertices += len(vertices)
        wf_hits = []
        for hit in hits:
            if hit.layer < 4:
                wf_hits.append(hit)
        if len(wf_hits) == 0:
            continue
        for vertex in vertices:
            layer_ells = Ellipses(vertex, tracks)
            for hit in wf_hits:
                layer = hit.layer
                if hit.layer < 2 and layer_ells[layer](hit.x, hit.y): # Wall
                    n_vetoed +=1
                    break
                elif layer_ells[layer](hit.x,hit.z): # Floor
                    n_vetoed +=1
                    break
print("number of vertices:", nVertices)
print("number vetoed:", n_vetoed)

number of vertices: 4389
number vetoed: 2628


## Survival Rate: Event Level ##

In [8]:
n_survived = 0
nEvents = 0
for f in pathList:
    events=joblib.load(f)
    file_hits = events["hits"]
    file_tracks = events["tracks"]
    file_vertices = events["vertices"]
    for i in range(len(file_hits)):
        hits= file_hits[i]
        tracks = file_tracks[i]
        vertices = file_vertices[i]
        if len(vertices) == 0:
            continue
        nEvents += 1
        wf_hits = []
        for hit in hits:
            if hit.layer < 4:
                wf_hits.append(hit)
        survived = True
        for vertex in vertices:
            layer_ells = Ellipses(vertex, tracks)
            for hit in wf_hits:
                layer = hit.layer
                if hit.layer < 2 and layer_ells[layer](hit.x, hit.y): # Wall
                    survived = False
                    break
                elif layer_ells[layer](hit.x,hit.z): # Floor
                    survived = False
                    break
        if survived:
            n_survived += 1
            
print("number of Events:", nEvents)
print("number surviving:", n_survived)

number of vertices: 3686
number vetoed: 1455


# Signal #

In [11]:
data_top_dir = f"/home/owhgabri/My_GitHub/data/Reconstruction/LLP"
pathList=[]

for rootFile, dirs, files in os.walk(data_top_dir):
    for filename in files:
        if ".pkl" in filename:
            pathList.append(os.path.join(rootFile, filename))


print(len(pathList))

5


## Survival Rate: Vertex Level ##

In [9]:
n_vetoed = 0
nVertices = 0
for f in pathList:
    events=joblib.load(f)
    file_hits = events["hits"]
    file_tracks = events["tracks"]
    file_vertices = events["vertices"]
    for i in range(len(file_hits)):
        hits= file_hits[i]
        tracks = file_tracks[i]
        vertices = file_vertices[i]
        if len(vertices) == 0:
            continue
        nVertices += len(vertices)
        wf_hits = []
        for hit in hits:
            if hit.layer < 4:
                wf_hits.append(hit)
        if len(wf_hits) == 0:
            continue
        for vertex in vertices:
            layer_ells = Ellipses(vertex, tracks)
            for hit in wf_hits:
                layer = hit.layer
                if hit.layer < 2 and layer_ells[layer](hit.x, hit.y): # Wall
                    n_vetoed +=1
                    break
                elif layer_ells[layer](hit.x,hit.z): # Floor
                    n_vetoed +=1
                    break
print("number of vertices:", nVertices)
print("number vetoed:",n_vetoed)

KeyboardInterrupt: 

## Survival Rate: Event Level ##

In [12]:
n_survived = 0
nEvents = 0
for f in pathList:
    events=joblib.load(f)
    file_hits = events["hits"]
    file_tracks = events["tracks"]
    file_vertices = events["vertices"]
    for i in range(len(file_hits)):
        hits= file_hits[i]
        tracks = file_tracks[i]
        vertices = file_vertices[i]
        if len(vertices) == 0:
            continue
        nEvents += 1
        wf_hits = []
        for hit in hits:
            if hit.layer < 4:
                wf_hits.append(hit)
        survived = True
        for vertex in vertices:
            layer_ells = Ellipses(vertex, tracks)
            for hit in wf_hits:
                layer = hit.layer
                if hit.layer < 2 and layer_ells[layer](hit.x, hit.y): # Wall
                    survived = False
                    break
                elif layer_ells[layer](hit.x,hit.z): # Floor
                    survived = False
                    break
        if survived:
            n_survived += 1
            
print("number of Events:", nEvents)
print("number surviving:", n_survived)

number of vertices: 46153
number vetoed: 25096
